In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import influxdb_client
import scipy

from influxdb_client.client.write_api import SYNCHRONOUS

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

warnings.simplefilter("ignore", MissingPivotFunction)

# configure influxdb client
ORG = "b4mad"
TOKEN = os.environ.get(
    "INFLUXDB_TOKEN",
    "citqAMr66LLb25hvaaZm2LezOc88k2ocOFJcJDR6QB-RmLJa_-sAr9kYB4vSFYaz8bt26lm7SokVgpQKdgKFKA==",
)
URL = "https://telemetry.b4mad.racing/"

# and create the client and a quary api
client = influxdb_client.InfluxDBClient(url=URL, token=TOKEN, org=ORG)
query_api = client.query_api()

session = 1669917645
lap = 2
game = "rFactor2"
car = "Ferrari F2004"
track = "Silverstone"

In [7]:
query = f"""
from(bucket: "racing")
|> range(start: -10y, stop: now())
|> filter(fn: (r) => r["_measurement"] == "laps_cc")
|> filter(fn: (r) => r["SessionId"] == "{session}")
// |> filter(fn: (r) => r["GameName"] == "{game}")
// |> filter(fn: (r) => r["CarModel"] == "{car}")
// |> filter(fn: (r) => r["TrackCode"] == "{track}")
|> filter(fn: (r) => r["CurrentLap"] == "{lap}")
|> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
|> sort(columns: ["_time"], desc: false)
"""
df = query_api.query_data_frame(org=ORG, query=query)

In [10]:
# show only the CurrentLapTime and DistanceOnTrack column
df[["CurrentLapTime", "DistanceRoundTrack", "CurrentLapTime"]]

,CurrentLapTime,DistanceRoundTrack,CurrentLapTime
0,0.095,4.797549,0.095
1,0.095,4.797549,0.095
2,0.352,22.296200,0.352
3,0.352,22.296200,0.352
4,0.607,39.488213,0.607
...,...,...,...
1124,122.400,5774.847000,122.400
1125,122.660,5792.144000,122.660
1126,122.660,5792.144000,122.660
1127,0.007,5807.000000,0.007
